In [1]:
# Importing required items
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Confirm the connection to the GPU with tensorflow
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul  8 20:09:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    40W / 300W |    596MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Determine the available RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
# Defining the training and testing directories
training_data_dir = '/content/drive/MyDrive/images/train'
test_data_dir = '/content/drive/MyDrive/images/test'

In [15]:
# Defining hyperparameters
img_width, img_height = 150, 150
batch_size = 15
epochs = 30
num_classes = 12  # Number of different types in dataset

In [16]:
# Defining data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [17]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [18]:
# Defining training generator
train_generator = train_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4750 images belonging to 12 classes.


In [19]:
# Defining the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [20]:
# Adjusting learning rate and change optimization algorithm
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [21]:
# Compiling the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [22]:
# Optimizing and improving memory usage
tf.config.optimizer.set_jit(True)

In [23]:
# Training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
)

Epoch 1/30
316/316 [==============================] - 97s 297ms/step - loss: 2.0277 - accuracy: 0.3086
Epoch 2/30
316/316 [==============================] - 92s 291ms/step - loss: 1.4392 - accuracy: 0.5073
Epoch 3/30
316/316 [==============================] - 93s 294ms/step - loss: 1.1842 - accuracy: 0.6055
Epoch 4/30
316/316 [==============================] - 90s 284ms/step - loss: 1.0531 - accuracy: 0.6448
Epoch 5/30
316/316 [==============================] - 88s 279ms/step - loss: 0.9419 - accuracy: 0.6887
Epoch 6/30
316/316 [==============================] - 89s 282ms/step - loss: 0.8479 - accuracy: 0.7130
Epoch 7/30
316/316 [==============================] - 90s 284ms/step - loss: 0.7703 - accuracy: 0.7350
Epoch 8/30
316/316 [==============================] - 94s 298ms/step - loss: 0.7495 - accuracy: 0.7430
Epoch 9/30
316/316 [==============================] - 90s 285ms/step - loss: 0.6876 - accuracy: 0.7728
Epoch 10/30
316/316 [==============================] - 89s 282ms/step - l

In [24]:
# Saving the trained model
model.save('project4_model.h5')

In [25]:
# Loading the trained model
model = tf.keras.models.load_model('project4_model.h5')

In [26]:
# Creating an ImageDataGenerator for the testing dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [27]:
# Loading the filenames of the images in the testing dataset
test_filenames = os.listdir(test_data_dir)

In [28]:
# Creating a dataframe with the filenames
test_df = pd.DataFrame({'filename': test_filenames})

In [29]:
# Generating the test data generator from the dataframe
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_data_dir,
    x_col='filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 794 validated image filenames.


In [30]:
# Predicting the classes for the test images
predictions = model.predict(test_generator)

53/53 [==============================] - 6s 104ms/step


In [31]:
# Maping predicted class indices to class labels
class_labels = list(train_generator.class_indices.keys())
predicted_classes = [class_labels[prediction.argmax()] for prediction in predictions]

In [32]:
# Printing the predicted classes
print(predicted_classes)

['Shepherds Purse', 'Common Chickweed', 'Sugar beet', 'Sugar beet', 'Sugar beet', 'Sugar beet', 'Small-flowered Cranesbill', 'Loose Silky-bent', 'Small-flowered Cranesbill', 'Small-flowered Cranesbill', 'Common Chickweed', 'Cleavers', 'Loose Silky-bent', 'Fat Hen', 'Maize', 'Loose Silky-bent', 'Loose Silky-bent', 'Maize', 'Fat Hen', 'Fat Hen', 'Loose Silky-bent', 'Common Chickweed', 'Scentless Mayweed', 'Common Chickweed', 'Small-flowered Cranesbill', 'Sugar beet', 'Scentless Mayweed', 'Common Chickweed', 'Scentless Mayweed', 'Sugar beet', 'Common Chickweed', 'Common Chickweed', 'Sugar beet', 'Charlock', 'Loose Silky-bent', 'Loose Silky-bent', 'Maize', 'Shepherds Purse', 'Scentless Mayweed', 'Loose Silky-bent', 'Maize', 'Maize', 'Cleavers', 'Sugar beet', 'Small-flowered Cranesbill', 'Common Chickweed', 'Sugar beet', 'Cleavers', 'Common Chickweed', 'Sugar beet', 'Loose Silky-bent', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Common Chickweed', 'Common wheat', 'Shepherds Purse', 'Cl

In [33]:
# Adding the predicted classes to the dataframe
test_df['predictions'] = predicted_classes
test_df.head()

,filename,predictions
0,0885e7690.png,Shepherds Purse
1,0d117d910.png,Common Chickweed
2,03e322a29.png,Sugar beet
3,043449b0b.png,Sugar beet
4,007b3da8b.png,Sugar beet
